In [1]:
!pip install crewai --quiet
!pip install 'crewai[tools]' --quiet
!pip install langchain_groq --quiet
!pip install agentops --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.4/170.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
import os
import agentops
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq

In [15]:
agentops.init('55caccb5-5715-4f48-94c3-5d9ae5e2e2b9', tags=["faq-prompt-grading", "AgentOPs_with_genAI_story"])

os.environ["SERPER_API_KEY"] = "9fff96a653dfa142001dc1fa54ff07d71270ca07"  # serper.dev API key
print(os.environ["SERPER_API_KEY"])

os.environ['GROQ_API_KEY'] = "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"
llm = ChatGroq(model="gemma-7b-it", groq_api_key=os.environ['GROQ_API_KEY'])
print(llm.invoke("hi"))

search_tool = SerperDevTool()

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=000b32c4-47ed-472c-b542-4770480dd7f6
INFO:agentops:Session Replay: https://app.agentops.ai/drilldown?session_id=000b32c4-47ed-472c-b542-4770480dd7f6


9fff96a653dfa142001dc1fa54ff07d71270ca07
content="Hi! 👋 It's great to hear from you. What can I do for you today? 😊" response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 15, 'total_tokens': 37, 'completion_time': 0.023723212, 'prompt_time': 0.017965063, 'queue_time': None, 'total_time': 0.041688275}, 'model_name': 'gemma-7b-it', 'system_fingerprint': 'fp_7d8efeb0b1', 'finish_reason': 'stop', 'logprobs': None} id='run-e0498a77-4110-48f0-8199-ddde2f9db19d-0' usage_metadata={'input_tokens': 15, 'output_tokens': 22, 'total_tokens': 37}


In [16]:
prompt_evaluator = Agent(
    role="Prompt Evaluator",
    goal='Evaluate the quality and completeness of FAQ generation prompts for {device_type}',
    backstory=(
        "As an expert in prompt engineering with extensive experience in consumer products, "
        "you have a keen eye for detail and a deep understanding of what makes an effective FAQ prompt. "
        "Your evaluations are thorough, considering both the technical aspects and the user-centric approach."
    ),
    llm=llm,
    max_iter=5,
    tools=[search_tool],
    allow_delegation=True,
    verbose=True
)



grader = Agent(
    role="Grader",
    goal='Assign final grades for FAQ prompts based on evaluations and guideline adherence for {device_type}',
    verbose=True,
    backstory=(
        "As an experienced teaching assistant with a background in NLP and prompt engineering, "
        "you have a holistic understanding of what makes an effective FAQ prompt. "
        "Your grading is fair, consistent, and takes into account both technical accuracy and practical effectiveness."
    ),
    tools=[search_tool],
    llm=llm,
    max_iter=5,
    allow_delegation=False
)

In [17]:
evaluation_task = Task(
    description=(
        "Evaluate the quality and completeness of the submitted FAQ prompt for {device_type}. "
        "Consider aspects such as coverage of key features, addressing potential user concerns, "
        "and overall effectiveness in generating comprehensive FAQs."
    ),
    expected_output='A detailed evaluation report with specific strengths and areas for improvement.',
    tools=[search_tool],
    agent=prompt_evaluator
)



grading_task = Task(
    description=(
        "Assign a final grade for the FAQ prompt based on the evaluation and guideline adherence reports. "
        "Consider both the technical aspects and the practical effectiveness of the prompt for {device_type}."
    ),
    expected_output='A final grade (out of 100) with a brief justification for the score.',
    tools=[search_tool],
    agent=grader,
    async_execution=False,
    output_file="final_grade.txt"
)

In [18]:
crew = Crew(
    agents=[prompt_evaluator, grader],
    tasks=[evaluation_task, grading_task],
    process=Process.sequential,
    max_rpm=3,
    cache=True
)

In [19]:
submission = """
You are an intelligent assistant designed to compile comprehensive FAQs for a new Product or Service. Your task is to anticipate common queries covering various potential aspects that interested people would enquire about.
Guidelines:
1. Ask for the product's/service's key features and benefits.
2. Length: Aim for a concise yet informative response for each question.
3. Focus: Address concerns related to functionality, compatibility with other devices, security measures, and customer support. Include details on setup, usage scenarios, and any unique selling points.
4. Tone and Style: Ensure a friendly and informative tone, suitable for a consumer audience.
5. Content: Provide factual answers with clarity, avoiding technical jargon unless necessary. Always confirm specifics before proceeding with each FAQ generation.
"""

result = crew.kickoff(inputs={'device_type': "smart home device", 'submission': submission})
print(result)



> Entering new CrewAgentExecutor chain...
## Detailed Evaluation Report: Smart Home Device FAQ Prompt

**Strengths:**

* **Comprehensive Feature Coverage:** The prompt covers a wide range of key features common to smart home devices, including connectivity, setup, automation, and security.
* **User-Centric Language:** The language is clear, concise, and easy for users to understand, with examples and metaphors to illustrate concepts.
* **Potential Concerns Addressed:** The prompt tackles common user concerns like privacy, security vulnerabilities, and compatibility issues.
* **Actionable Information:** The prompt provides actionable steps for resolving issues and accessing support resources.


**Areas for Improvement:**

* **Specificity and Context:** While the prompt covers various features, it lacks specifics on how to address certain scenarios, such as troubleshooting connectivity issues or changing automation schedules.
* **Visual Aids:** The prompt would benefit from the inclusi

🖇 AgentOps: This run's cost $0.00
INFO:agentops:This run's cost $0.00
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=000b32c4-47ed-472c-b542-4770480dd7f6
INFO:agentops:Session Replay: https://app.agentops.ai/drilldown?session_id=000b32c4-47ed-472c-b542-4770480dd7f6


**Grade: 85/100**

**Justification:**

The FAQ prompt demonstrates strong foundational elements, addressing key features, user concerns, and privacy/security issues. Its user-friendly language and examples are commendable. However, adding context-specific examples, visual aids, and a troubleshooting guide would significantly enhance its effectiveness.

**Action:**

- Integrate specific examples illustrating:
    - Connectivity troubleshooting with network diagrams and connection issues.
    - Automation scheduling adjustments with practical scenarios.
- Provide detailed steps for resolving privacy concerns related to data collection and sharing.
- Offer guidance on resolving compatibility issues with other smart home devices.
- Include diagrams for network connections, device configurations, and automation workflows.
- Use screenshots to demonstrate setup processes and troubleshooting steps.
- Develop a dedicated section with common problems and their resolutions.
- Provide visual repr